## read data

In [1]:
import os

In [28]:
os.listdir('./Dataset/')

['.DS_Store',
 'm5-forecasting-accuracy',
 'm5-forecasting-accuracy.zip',
 'Trading_at_Close']

In [29]:
data_path = './Dataset/Trading_at_Close/'

In [30]:
os.listdir(data_path)

['.DS_Store', 'test.csv', 'train.csv', 'test.csv.zip', 'train.csv.zip']

In [69]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from numpy.random import rand
from numpy.random import choice
import random

In [50]:
df_train = pd.read_csv(f'{data_path}/train.csv')
df_test = pd.read_csv(f'{data_path}/test.csv')

In [51]:
df_train.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,target,time_id,row_id
0,0,0,0,3180602.69,1,0.999812,13380276.64,NaN,NaN,0.999812,60651.50,1.000026,8493.03,1.0,-3.029704,0,0_0_0
1,1,0,0,166603.91,-1,0.999896,1642214.25,NaN,NaN,0.999896,3233.04,1.000660,20605.09,1.0,-5.519986,0,0_0_1
2,2,0,0,302879.87,-1,0.999561,1819368.03,NaN,NaN,0.999403,37956.00,1.000298,18995.00,1.0,-8.389950,0,0_0_2
3,3,0,0,11917682.27,-1,1.000171,18389745.62,NaN,NaN,0.999999,2324.90,1.000214,479032.40,1.0,-4.010200,0,0_0_3
4,4,0,0,447549.96,-1,0.999532,17860614.95,NaN,NaN,0.999394,16485.54,1.000016,434.10,1.0,-7.349849,0,0_0_4


In [52]:
df_test.head()

,stock_id,date_id,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,ask_size,wap,time_id,row_id,currently_scored
0,0,478,0,3753451.43,-1,0.999875,11548975.43,NaN,NaN,0.999875,22940.00,1.000050,9177.60,1.0,26290,478_0_0,False
1,1,478,0,985977.11,-1,1.000245,3850033.97,NaN,NaN,0.999940,1967.90,1.000601,19692.00,1.0,26290,478_0_1,False
2,2,478,0,599128.74,1,1.000584,4359198.25,NaN,NaN,0.999918,4488.22,1.000636,34955.12,1.0,26290,478_0_2,False
3,3,478,0,2872317.54,-1,0.999802,27129551.64,NaN,NaN,0.999705,16082.04,1.000189,10314.00,1.0,26290,478_0_3,False
4,4,478,0,740059.14,-1,0.999886,8880890.78,NaN,NaN,0.999720,19012.35,1.000107,7245.60,1.0,26290,478_0_4,False


In [53]:
far_price_mean = df_train.groupby('stock_id')['far_price'].mean()
near_price_mean = df_train.groupby('stock_id')['near_price'].mean()

In [54]:
df_train.loc[df_train.far_price.isna(),'far_price'] = df_train.loc[df_train.far_price.isna()]['stock_id'].map(far_price_mean)
df_train.loc[df_train.near_price.isna(),'near_price'] = df_train.loc[df_train.near_price.isna()]['stock_id'].map(near_price_mean)

In [55]:
# fill na Test , avg from Train

In [56]:
df_test.loc[df_test.far_price.isna(),'far_price'] = df_test.loc[df_test.far_price.isna()]['stock_id'].map(far_price_mean)

df_test.loc[df_test.near_price.isna(),'near_price'] = df_test.loc[df_test.near_price.isna()]['stock_id'].map(near_price_mean)

In [57]:
df_train.isna().mean()

stock_id                   0.000000
date_id                    0.000000
seconds_in_bucket          0.000000
imbalance_size             0.000042
imbalance_buy_sell_flag    0.000000
reference_price            0.000042
matched_size               0.000042
far_price                  0.000000
near_price                 0.000000
bid_price                  0.000042
bid_size                   0.000000
ask_price                  0.000042
ask_size                   0.000000
wap                        0.000042
target                     0.000017
time_id                    0.000000
row_id                     0.000000
dtype: float64

In [58]:
df_train.dropna(inplace=True)

In [59]:
df_test.fillna(0, inplace=True)

In [109]:
import random
import copy
import statistics
from sklearn.metrics import make_scorer # to define custom loss function 
import sklearn

## Loss Functions

In [122]:
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

custom_loss_rmse =lambda y_true, y_pred: (mean_squared_error(y_true,y_pred))**0.5
custom_loss_mse = lambda y_true, y_pred: mean_squared_error(y_true,y_pred)
custom_loss_mae = lambda y_true, y_pred:mean_absolute_error(y_true,y_pred)
custom_loss_mape = lambda y_true, y_pred:mean_absolute_percentage_error(y_true,y_pred)
r2 = lambda y_true, y_pred:r2_score(y_true,y_pred)
loss_map = {'rmse':custom_loss_rmse, 'mse':custom_loss_mse,'mape':custom_loss_mape, 'mae':custom_loss_mae,'r2':r2,'max_error':'max_error'}

# Feature Selection with Genetic Algorithm

In [123]:
## Init Population

In [124]:
def init_population(population_size,num_features):
    population = []
    max_features = round(0.95*num_features)
    for i in range(population_size):
        individual = [True]*random.randrange(max_features)
        f_c = [False]*(num_features-(len(individual)))
        individual.extend(f_c)
        random.shuffle(individual)
      #individual = list(choice([True, False], size=num_features))
        while individual in population:
            individual = [True]*random.randrange(max_features)
            f_c = [False]*(num_features-(len(individual)))
            individual.extend(f_c)
            random.shuffle(individual)
      #Edge Case: if all genes are 0, then we will make any one gene as 1 
      # ( this will ensure atleast one feature in solution)
        if(sum(individual)==0):
            position = random.randrange(num_features)
            individual[position] = True
        population.append(individual)
    
    return population

In [125]:
def get_fitness(population,X,y,estimator,loss):
    fitness_values = []
    for individual in population:
        ix = [i for i, x in enumerate(individual) if x]
        X_new = X.iloc[:, ix].copy()
        y = y.copy()
        individual_fitness = calculate_fitness(X_new,y,estimator,loss)
        fitness_values.append(individual_fitness)
    return fitness_values

In [126]:
from sklearn.model_selection import TimeSeriesSplit
def calculate_fitness(X,y,estimator=RandomForestRegressor(random_state=42), loss = 'rmse'):
    """X is train data subset with selected feature use custome scorer for regression to by pass higher the better condition
      default loss is MAE """
    X = X.copy()
    y = y.copy()
    #score = make_scorer(loss_map.get(loss,custom_loss_mae), greater_is_better=False)
    #score is -ve for CV optimization 
    # scores in cross_val_score 
    tscv = TimeSeriesSplit(n_splits = 5)
    scores = []
    for train_index, test_index in tscv.split(X):
        cv_train, cv_test = X.iloc[train_index], X.iloc[test_index]
        #print(cv_test.shape)
        y_train,y_test = y.iloc[train_index],y.iloc[test_index]
        model = estimator.fit(cv_train,y_train)
        predictions = model.predict(cv_test)
        scores.append((loss_map[loss](y_test, predictions)))
      # scorer = loss_map.get(loss,'neg_mean_absolute_error')
      # scores = cross_val_score(estimator,X,y,scoring=scorer,n_jobs=-1,cv=10) 
    return 1/np.mean((scores))

In [163]:
def select_parents(population,n_parents, fitness_values):
    """steps for Roulette Wheel Selection
    [Sum] Calculate the sum of all chromosome fitnesses in population - sum S.
    [Select] Generate random number from the interval (0,S) - r.
    [Loop] Go through the population and sum the fitnesses from 0 - sum s. When the sum s is greater then r, stop and return the chromosome where you are.
    the step 1 is performed only once for each population.
    1. Calculate S = the sum of a finesses.
    2. Generate a random number R between 0 and S.
    3. Starting from the top of the population, keep adding the fitnesses to the partial sum P, till P less than R.
    4. The individual for which P exceeds R is the chosen individual. 
    """
    parents = []
    # Elitism pick top parents as is for next generation
    ind = pd.Series(fitness_values).argsort()[::-1]
    elit = round(n_parents*0.01)
    indx = ind[:elit]
    elit_parents = []
    for i in indx:
        elit_parents.append(population[i])
    parents.extend(elit_parents)
    total = sum(fitness_values) # S
    norm_fitness_values = pd.Series(fitness_values)/total # relative fitness
    cumulative_fitness = norm_fitness_values.cumsum()
    population_size = len(population)
    for count in range(round(n_parents*0.9)):
        random_number = random.uniform(0, 1) # R random number from 0,S or 1 in case on normalized scores
        individual_number = 0
        for score in cumulative_fitness:
            if(random_number <= score):
                id_ = population[individual_number]
                parents.append(id_)
                break
            individual_number+=1   
        
    return parents,elit_parents

In [164]:
def one_point_crossover(parents,probability):
    """In this one-point crossover, a random crossover point is selected and the tails of its two parents are swapped to get new off-springs """
    chromosome_len = len(parents[0])
    crossover_population = []
    no_of_pairs = round(len(parents)/2)
    for num in range(no_of_pairs):
        length = len(parents)
        parent1_index, parent2_index = random.sample(range(length), k=2)
        parent1 = parents[parent1_index]
        parent2 = parents[parent2_index]
        if random.uniform(0,1)<probability: 
            crossover_point = random.randrange(chromosome_len)
            child1 = parent1[:crossover_point]
            child1.extend(parent2[crossover_point:])
            child2 = parent1[:crossover_point]
            child2.extend(parent2[crossover_point:])
            parents.remove(parent1)
            parents.remove(parent2)
            if sum(child1)>0:
                crossover_population.append(child1)
            if sum(child2)>0:
                crossover_population.append(child2)
        else:
            pass

    if(len(parents)>0):
        crossover_population.extend(parents)
    return crossover_population

In [165]:
def uniform_crossover(parents,probability):
    """In a uniform crossover, we don’t divide the chromosome into segments, rather we treat each gene separately. In this, we essentially flip a coin for each chromosome to decide whether or not it’ll be included in the off-spring. We can also bias the coin to one parent, to have more genetic material in the child from that parent.
  Usually, a probability is assigned to this process, indicating the chance of crossover for a given pair. Crossover is a high probability event and is assigned an optimum probability between 0.65–0.80. Here, I have used a probability of 0.78.
  """
    random.shuffle(parents)
    crossover_population = []
    chromosome_len = len(parents[0])
    no_of_pairs = round(len(parents)/2)
    for num in range(no_of_pairs):
        length = len(parents)
        parent1_index, parent2_index = random.sample(range(length), k=2)
        parent1 = parents[parent1_index]
        parent2 = parents[parent2_index]
        if random.uniform(0,1)<=probability: 
            swap = [rand()<0.5 for i in range(chromosome_len)]
            child1 = [parent1[i] if j else parent2[i] for i,j in enumerate(swap)]
            child2 = [parent2[i] if j else parent1[i] for i,j in enumerate(swap)]
            parents.remove(parent1)
            parents.remove(parent2)
            if sum(child1)>0:
                crossover_population.append(child1)
            if sum(child2)>0:
                crossover_population.append(child2)
        else:
            pass
    if(len(parents)>0):
        crossover_population.extend(parents)
    return crossover_population


In [166]:
def mutation(crossover_population):
    for individual in crossover_population:
        index_1, index_2 = random.sample(range(len(individual)), k=2)
        #swapping the bits
        individual[index_1],individual[index_2] = individual[index_2],individual[index_1]
    return crossover_population

In [172]:
def GA_feature_selection(X,y,n_gen, mutation_prob=0.005, estimator=RandomForestRegressor(random_state=42), loss='mae'):
    best_fitness_chromosome = {}
    X = X.copy()
    y = y.copy()
    num_features = X.shape[1]
    population_size = round(2*num_features)
    n_parents = round(population_size*3/4)
    # init population
    population = init_population(population_size,num_features)
    print("generated population of size",len(population))
    # calculate fitness value of population
    fitness_values = get_fitness(population,X,y,estimator,loss)
    parents,elit_parents = select_parents(population,n_parents, fitness_values)
    best_fitness_chromosome['init'] = population[pd.Series(fitness_values).argmax()]
    crossover_population = uniform_crossover(parents,0.8)
    #crossover_population = one_point_crossover(parents,0.85)
    population = crossover_population
    p = random.uniform(0,1)
    
    if p < mutation_prob:
        mutated_population = mutation(crossover_population)
        population = mutated_population
    fitness_values = get_fitness(population,X,y,estimator,loss)
    best_fitness_chromosome['gen0'] = population[pd.Series(fitness_values).argmax()]
    variance_of_population = statistics.variance(1/pd.Series(fitness_values))
    print(f"starting variance is={variance_of_population}",)
    gen = 1
    best_chromosome_fitness = {}
    elit_pop = {}
    threshold = 0.001
  
    #for i in range(n_gen):
    while (variance_of_population > threshold) and gen < n_gen  :
        print(f'generation-{gen}')
        parents,elit_parents = select_parents(population,n_parents,fitness_values)
        elit_pop[f'gen{gen}'] = list(pd.DataFrame(elit_parents).T.sum(axis=1))
        crossover_population = uniform_crossover(parents,0.7)
        #crossover_population = one_point_crossover(parents,0.85)
        population = crossover_population
        p = random.uniform(0,1)
        if(p<=mutation_prob): #mutation prob here
            mutated_population = mutation(crossover_population)
            population = mutated_population
        fitness_values = get_fitness(population,X,y,estimator,loss)
        best_fiteness =1/fitness_values[pd.Series(fitness_values).argmax()]
        best_chro = population[pd.Series(fitness_values).argmax()]
        best_fitness_chromosome[f'gen{gen}'] = best_chro
        best_chromosome_fitness[f'gen{gen}'] = best_fiteness
        variance_of_population = statistics.variance(1/pd.Series(fitness_values))
        acc_score_population = np.mean(fitness_values)
        print(f"num_features in best chromosome = {sum(best_chro)} variance is= {variance_of_population}, avg {loss} = {round(1/acc_score_population)}, best = {round(best_fiteness)}")
        gen+=1
    best_features = []
    best_acc_score = 0
    avg_fitness = sum(fitness_values)/len(fitness_values)
    #avg_fitness_2 = sum(1/fitness_values)/len(fitness_values)
    avg_fitness_2 = 1/(avg_fitness)
    index = (pd.Series(fitness_values)).argmax()
    fitness_value = fitness_values[index]
  
    print(f'avg fitness is:{avg_fitness}, {avg_fitness_2}, best fitness is: {round(1/fitness_value,2)}')
   
    best_features = population[index]
    best_acc_score = fitness_value
  

    return best_features,best_acc_score#, best_fitness_chromosome,best_chromosome_fitness,elit_pop

In [132]:
df_train.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')

In [133]:
stock_id = random.choice(df_train['stock_id'].unique())

In [134]:
df_train_tmp =df_train[df_train['stock_id']==stock_id].copy()

In [158]:
## Train_ Validation split

In [159]:
from sklearn.model_selection import train_test_split

In [160]:
X_train, X_Val = train_test_split(df_train_tmp,test_size=0.2)

In [161]:
y_train = X_train['target']
X_train = X_train[['seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap']].copy()

v_val = X_Val['target']
X_Val = X_Val[['seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap']].copy()

In [ ]:
best_features,best_acc_score = GA_feature_selection(X_train,y_train,n_gen=20,estimator=RandomForestRegressor(random_state=57, n_estimators=100),loss='rmse')

generated population of size 24
starting variance is=0.2342611661237598
generation-1
num_features in best chromosome = 7 variance is= 0.2384320464879258, avg rmse = 8, best = 7
generation-2
num_features in best chromosome = 7 variance is= 0.23410031533019668, avg rmse = 8, best = 7
generation-3
num_features in best chromosome = 7 variance is= 0.11294582425505671, avg rmse = 8, best = 7
generation-4
num_features in best chromosome = 7 variance is= 0.23591343404217968, avg rmse = 8, best = 7
generation-5
num_features in best chromosome = 7 variance is= 0.026044438394346718, avg rmse = 8, best = 7
generation-6


In [143]:
print('Selected features:: ', X_train.columns[best_features], 'from list :',X_train.columns)

Selected features::  Index(['seconds_in_bucket', 'imbalance_size', 'matched_size', 'near_price',
       'bid_size', 'ask_size', 'wap'],
      dtype='object') from list : Index(['seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag',
       'reference_price', 'matched_size', 'far_price', 'near_price',
       'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap'],
      dtype='object')


In [147]:
selected_features = list(X_train.columns[best_features])

In [154]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [148]:
model = LinearRegression()
model.fit(X_train[selected_features], y_train)

LinearRegression()

In [153]:
y_pred = model.predict(X_test[selected_features])